In [1]:
import os
import logging
import sys

import openai
from dotenv import load_dotenv
from IPython.display import Markdown, display
from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer


load_dotenv()

logging.basicConfig(
    stream=sys.stdout,
    level=logging.DEBUG,
)

openai.api_key = os.environ["OPENAI_API_KEY"]

space_name = "policies_aa"
edge_types, rel_prop_names = ["relationship"], [
    "name"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

storage_context = StorageContext.from_defaults(
    persist_dir="../../data/index/policies/american_airlines",
    graph_store=graph_store
)

kg_index = load_index_from_storage(
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    verbose=True,
)

chat_store = SimpleChatStore()

chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=chat_store,
    chat_store_key="user1",
)

chat_engine = kg_index.as_chat_engine(memory=chat_memory)

DEBUG:llama_index.core.storage.kvstore.simple_kvstore:Loading llama_index.core.storage.kvstore.simple_kvstore from ../../data/index/policies/american_airlines/docstore.json.
DEBUG:fsspec.local:open file: /Users/pez/Projects/nlp_airline_project/notebooks/kg-llamaindex/../../data/index/policies/american_airlines/docstore.json
DEBUG:llama_index.core.storage.kvstore.simple_kvstore:Loading llama_index.core.storage.kvstore.simple_kvstore from ../../data/index/policies/american_airlines/index_store.json.
DEBUG:fsspec.local:open file: /Users/pez/Projects/nlp_airline_project/notebooks/kg-llamaindex/../../data/index/policies/american_airlines/index_store.json
DEBUG:llama_index.core.vector_stores.simple:Loading llama_index.core.vector_stores.simple from ../../data/index/policies/american_airlines/image__vector_store.json.
DEBUG:fsspec.local:open file: /Users/pez/Projects/nlp_airline_project/notebooks/kg-llamaindex/../../data/index/policies/american_airlines/image__vector_store.json
DEBUG:llama_in

In [2]:
question = "What is the fee for a checked bag on American Airlines?"
response = chat_engine.chat(question)
display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {response}</b>"))

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/pez/.local/share/virtualenvs/nlp_airline_project-fCxuAmfF/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'What is the fee for a checked bag on American Airlines?'}], 'model': 'gpt-3.5-turbo', 'stream': False, 'temperature': 0.1, 'tool_choice': 'auto', 'tools': [{'type': 'function', 'function': {'name': 'query_engine_tool', 'description': 'Useful for running a natural language query\nagainst a knowledge base and get back a natural language response.\n', 'parameters': {'type': 'object', 'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input']}}}]}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:connect_

<b>Q: What is the fee for a checked bag on American Airlines?<b>

<b>A: The fee for a checked bag on American Airlines varies depending on the destination. For travel within or between the U.S., Puerto Rico, and U.S. Virgin Islands, the 1st checked bag fee is $40 ($35 if paid online) and the 2nd checked bag fee is $45. For travel to or from Canada, Caribbean, Mexico, Central America, and Guyana, the 1st checked bag fee is $35 and the 2nd checked bag fee is $45.</b>

In [3]:
question_2 = "I'm flying economy class to Cuba."
response_2 = chat_engine.chat(question_2)
display(Markdown(f"<b>Q: {question_2}<b>\n\n<b>A: {response_2}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'What is the fee for a checked bag on American Airlines?'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_80C7bkLBahaOg7j5yujREmLV', 'function': {'arguments': '{"input":"What is the fee for a checked bag on American Airlines?"}', 'name': 'query_engine_tool'}, 'type': 'function'}]}, {'role': 'tool', 'content': 'The fee for a checked bag on American Airlines varies depending on the destination. For travel within or between the U.S., Puerto Rico, and U.S. Virgin Islands, the 1st checked bag fee is $40 ($35 if paid online) and the 2nd checked bag fee is $45. For travel to or from Canada, Caribbean, Mexico, Central America, and Guyana, the 1st checked bag fee is $35 and the 2nd checked bag fee is $45.', 'name': 'query_engine_tool', 'tool_call_id': 'call_80C7bkLBahaOg7j5yujREmLV'}, {'role': 'assistant', 'content':

<b>Q: I'm flying economy class to Cuba.<b>

<b>A: For a flight to Cuba in economy class on American Airlines, the fee for the 1st checked bag is $30 for Basic Economy tickets issued on or after September 20, 2023. There is no 1st bag fee for Main Cabin tickets issued on or after September 20, 2023.

The baggage policy for American Airlines flights to Cuba in economy class allows each passenger to check up to 2 bags with a maximum weight of 70 lbs (32 kgs) per bag. Passengers are also permitted to bring 1 carry-on bag and 1 personal item. Additional fees apply for extra bags depending on the season and the class of ticket purchased.</b>

In [4]:
question_2 = "What about on United Airlines?"
response_2 = chat_engine.chat(question_2)
display(Markdown(f"<b>Q: {question_2}<b>\n\n<b>A: {response_2}</b>"))

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'What is the fee for a checked bag on American Airlines?'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_80C7bkLBahaOg7j5yujREmLV', 'function': {'arguments': '{"input":"What is the fee for a checked bag on American Airlines?"}', 'name': 'query_engine_tool'}, 'type': 'function'}]}, {'role': 'tool', 'content': 'The fee for a checked bag on American Airlines varies depending on the destination. For travel within or between the U.S., Puerto Rico, and U.S. Virgin Islands, the 1st checked bag fee is $40 ($35 if paid online) and the 2nd checked bag fee is $45. For travel to or from Canada, Caribbean, Mexico, Central America, and Guyana, the 1st checked bag fee is $35 and the 2nd checked bag fee is $45.', 'name': 'query_engine_tool', 'tool_call_id': 'call_80C7bkLBahaOg7j5yujREmLV'}, {'role': 'assistant', 'content':

<b>Q: What about on United Airlines?<b>

<b>A: For a flight to Cuba in economy class on United Airlines, the fee for the 2nd checked bag is $150.

The baggage policy for United Airlines flights to Cuba in economy class allows each passenger to check up to 2 bags with a maximum weight of 70 lbs per bag. Passengers are also permitted 1 carry-on bag and 1 personal item. Additional fees apply for extra bags depending on the travel dates and destination, with seasonal exceptions and varying charges for second bags.</b>

In [ ]:
question = "What is the fee for a checked bag on United Airlines on an economy class flight to Cuba?"
response = chat_engine.chat(question)
display(Markdown(f"<b>Q: {question}<b>\n\n<b>A: {response}</b>"))